<a href="https://colab.research.google.com/github/jinad-collab/Pretrained-model-text-classification/blob/main/%22Jinad4success_hospitality_customer_agent%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL NEEDED LIBRARIES

In [ ]:
!pip install -q transformers datasets accelerate huggingface_hub


LOAD DATASET

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MakTek/Customer_support_faqs_dataset")


FORMAT THE DATA

In [ ]:
def format_example(example):
    return {
        "input_text": f"Answer this customer support question:\n{example['question']}",
        "target_text": example["answer"]
    }

train_data = dataset["train"].map(format_example)


TOKENIZER + THE MODEL

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


TOKENIZE DATA

In [ ]:
def tokenize(batch):
    model_inputs = tokenizer(
        batch["input_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    labels = tokenizer(
        batch["target_text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data = train_data.map(tokenize, batched=True)
tokenized_data = tokenized_data.remove_columns(train_data.column_names)


TRAINING ARGUMENTS

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./hospitality-fast-agent",
    per_device_train_batch_size=8,
    learning_rate=3e-4,
    num_train_epochs=1,
    logging_steps=50,
    save_steps=500,
    save_total_limit=1,
    report_to="none",
    fp16=False,
    bf16=False
)


TRAIN THE MODEL ITSELF

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data
)

trainer.train()


Step,Training Loss


TrainOutput(global_step=25, training_loss=10.770450439453125, metrics={'train_runtime': 24.4027, 'train_samples_per_second': 8.196, 'train_steps_per_second': 1.024, 'total_flos': 9294525235200.0, 'train_loss': 10.770450439453125, 'epoch': 1.0})

TESTING IT, TO CHECK IF THERE ISNT ANY BUG, OR ANY STUFF LIKE THAT

In [ ]:
def ask_agent(question):
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=50
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

ask_agent("How do I contact customer support?")

Customers, please contact them directly.


WOW, IT WORKED!!!!!💥😘😎  ☝



PUSHING IT TO HUGGING FACE💥 ALHAMDULILAH AWWAL IS MOVING

In [ ]:
model.push_to_hub("jinad4success/hospitality-fast-agent")
tokenizer.push_to_hub("jinad4success/hospitality-fast-agent")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...t-agent/model.safetensors:   5%|5         | 16.7MB /  308MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...y-fast-agent/spiece.model: 100%|##########|  792kB /  792kB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/jinad4success/hospitality-fast-agent/commit/8df5f40760c889ff09584092ac976f787ed3ac50', commit_message='Upload tokenizer', commit_description='', oid='8df5f40760c889ff09584092ac976f787ed3ac50', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jinad4success/hospitality-fast-agent', endpoint='https://huggingface.co', repo_type='model', repo_id='jinad4success/hospitality-fast-agent'), pr_revision=None, pr_num=None)